<a href="https://colab.research.google.com/github/fatimasr68/Emotion-Prediction-DNN/blob/master/Regression/IntraSub_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Imports**

In [ ]:
import tensorflow as tf
import scipy.io as sio
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import keras
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.layers import Dense, Input, LeakyReLU, Dropout
from keras import layers
from keras import optimizers, activations, regularizers, initializers
from keras.regularizers import L1L2
from keras.models import Model, Sequential
from keras.layers.recurrent import LSTM, GRU
from sklearn.metrics  import classification_report
import time

# **setup seed for Numpy**

In [ ]:
seed = 8000

np.random.seed(seed)
np.random.RandomState(seed)
tf.random.set_seed(seed)

# **Load Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/3wGqwOYGukUy6uoIpA1KC8ROiwhFz9YvqOWJGyheG-T6fs1j5kFnxKM
Mounted at /content/drive


In [ ]:
X0  = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/sub-000_feature.csv', header=None).to_numpy()
X1  = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/sub-001_feature.csv', header=None).to_numpy()
X2  = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/sub-002_feature.csv', header=None).to_numpy()
X3  = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/sub-003_feature.csv', header=None).to_numpy()
X4  = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/sub-004_feature.csv', header=None).to_numpy()
X5  = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/sub-005_feature.csv', header=None).to_numpy()
X6  = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/sub-006_feature.csv', header=None).to_numpy()
X7  = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/sub-007_feature.csv', header=None).to_numpy()
X8  = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/sub-008_feature.csv', header=None).to_numpy()
X9  = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/sub-009_feature.csv', header=None).to_numpy()
X10 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/sub-010_feature.csv', header=None).to_numpy()
X11 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/sub-011_feature.csv', header=None).to_numpy()
X12 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/sub-012_feature.csv', header=None).to_numpy()
X13 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/sub-013_feature.csv', header=None).to_numpy()
X14 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/sub-014_feature.csv', header=None).to_numpy()
X15 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/sub-015_feature.csv', header=None).to_numpy()
X16 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/sub-016_feature.csv', header=None).to_numpy()
X17 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/sub-017_feature.csv', header=None).to_numpy()
X18 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/sub-018_feature.csv', header=None).to_numpy()


Y = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/val_score.csv', header=None).to_numpy()


In [ ]:
X_List = [X0, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18]

In [ ]:
def scaler(X_all):

  #scaler = sklearn.preprocessing.MinMaxScaler()
  scaler = sklearn.preprocessing.StandardScaler()
  X_all = scaler.fit_transform(X_all)
  res = X_all.reshape(19, 90, 37130)
  return res.tolist()

def read_data(X_List, Y, x_index):  
  x_list = X_List.copy()
  y = Y.copy()

  y[y <= 4.99], y[y > 4.99] = 1, 0
  x_all_concatted = np.concatenate(x_list, axis=0)
  x_scaled = scaler(x_all_concatted)
  
  X_test = np.array(x_scaled.pop(x_index))
  X_train = np.concatenate(x_scaled)
  
  Y_test = y
  Y_train = np.concatenate((y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y), axis=0)
  Y_train = to_categorical(Y_train)
  Y_test = to_categorical(Y_test)

  X_test = np.expand_dims(X_test, axis=2)
  X_train = np.expand_dims(X_train, axis=2)

  return X_train, X_test, Y_train, Y_test

X_train, X_test, Y_train, Y_test = read_data(X_List, Y, 18)
# for i in range(18):
#   print([(X_train[i*90:(i*90+90),:] == X17).all()])

## **Create & Compile Model**

In [ ]:
n_feature = 37130   #37130 
[n_in, n_hidden1, n_hidden2, n_hidden3, n_output] = [n_feature, 64, 34, 0, 1] #64, 34  6000
timesteps = 1


class ClipConstraint(keras.constraints.Constraint):
  def __init__(self, clip_value):
    self.clip_value = clip_value
  def __call__(self, weights):
    return keras.backend.clip(weights, -self.clip_value, self.clip_value)

def create_model(l1 = 1e-5, l2 = 1e-5):

  clipC = ClipConstraint(0.05)
  #l1_l2 = keras.regularizers.l1_l2(l1=l1, l2=l2)
  
  model = tf.keras.models.Sequential()
  #model = Sequential()

  model.add(LSTM(units= n_hidden1,  
                dropout= 0.2,  
                recurrent_dropout= 0.7,  #0.7
                activation = 'relu',
                recurrent_activation = 'sigmoid',

                bias_regularizer = L1L2(l1= l1, l2= l2),
                kernel_regularizer = L1L2(l1= l1, l2= l2),
                activity_regularizer = L1L2(l1= l1, l2= l2),

                kernel_initializer = initializers.he_normal(),
                return_sequences= True,
                input_shape=(timesteps, n_in)))  


  #model.add(Dropout(0.1))

  model.add(LSTM(units= n_hidden2,  
                dropout= 0.2,  # 0.1,..., 0.5
                recurrent_dropout= 0.7,  #0.7
                activation = 'sigmoid',
                recurrent_activation = 'sigmoid',

                bias_regularizer = L1L2(l1= l1, l2= l2),
                kernel_regularizer = L1L2(l1= l1, l2= l2),
                activity_regularizer = L1L2(l1= l1, l2= l2),
                 
                kernel_initializer = initializers.orthogonal(),
                return_sequences= False))


  model.add(Dense(units= 2, 
                  activation= 'sigmoid'))





  return model




plot_model(create_model(), show_shapes= True)
create_model().summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 1, 64)             9521920   
_________________________________________________________________
lstm_3 (LSTM)                (None, 34)                13464     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 70        
Total params: 9,535,454
Trainable params: 9,535,454
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#plot_model(create_model(), show_shapes= True)

# **KFOLD ONE SUB**

In [ ]:
'''
from sklearn.model_selection import KFold, StratifiedKFold

CV_history = [] 

val_data_x = []
val_data_y = []

train_data_x = []
train_data_y = []

pred_val_tr = []
pred_val_ts = []

ACCs  = []
conf_mat = []

# scaler = sklearn.preprocessing.MinMaxScaler()
# X_5_fold = scaler.fit_transform(X1)
X_5_fold = X15.copy()

y = Y.copy()
y[y <= 4.99], y[y > 4.99] = 1, 0

class _callback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if (epoch + 1) % 50 == 0:
      self.model.optimizer.lr = self.model.optimizer.lr / 2

checkpoint_filepath = '/tmp/checkpoint'

fold_no = 0 
#StratifiedKFold
for train_index, val_index in StratifiedKFold(n_splits= 5, shuffle= True, random_state= seed).split(X_5_fold, y):

  x_tr, x_val = X_5_fold[train_index], X_5_fold[val_index]
  y_tr, y_val = y[train_index], y[val_index]



  fold_no += 1

  print('-------------------------------------------------------------------------------')
  print('Training for fold ', fold_no)
  print('-------------------------------------------------------------------------------')


  train_data_x.append(X_5_fold[train_index])
  train_data_y.append(y[train_index])
  val_data_x.append(X_5_fold[val_index])
  val_data_y.append(y[val_index])

  x_tr  = np.reshape(x_tr , (x_tr.shape[0] , 1, x_tr.shape[1]))
  x_val = np.reshape(x_val, (x_val.shape[0], 1, x_val.shape[1])) 

  model = None
  model = create_model()  #create_model(1e-5, 1e-5)
  
  model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
      filepath=checkpoint_filepath,
      save_weights_only=True,
      monitor='val_accuracy',
      verbose= 1,
      mode='max',
      save_best_only=True)
  
  model.compile(optimizer= keras.optimizers.Adam(1e-4),
              loss= keras.losses.categorical_crossentropy,
              metrics= ['accuracy'])
  
  y_tr = to_categorical(y_tr)
  y_val = to_categorical(y_val)

  results = model.fit(x_tr, y_tr, epochs= 200,
                          batch_size= 32,
                          verbose = 2,
                          callbacks= [model_checkpoint_callback],
                          validation_data = (x_val, y_val))

  # Updating to use the best model
  model.load_weights(checkpoint_filepath)

  pred_val_tr.append(model.evaluate(x_tr))
  pred_val_ts.append(model.evaluate(x_val))


  scores = model.evaluate(x_val, y_val)
  print("%s: %.2f" % (model.metrics_names[1], scores[1] ))   #ACC
  ACCs.append(scores[1])


  CV_history.append(results)
  print("======="*12, end="\n\n\n")
 
'''

'\nfrom sklearn.model_selection import KFold, StratifiedKFold\n\nCV_history = [] \n\nval_data_x = []\nval_data_y = []\n\ntrain_data_x = []\ntrain_data_y = []\n\npred_val_tr = []\npred_val_ts = []\n\nACCs  = []\nconf_mat = []\n\n# scaler = sklearn.preprocessing.MinMaxScaler()\n# X_5_fold = scaler.fit_transform(X1)\nX_5_fold = X15.copy()\n\ny = Y.copy()\ny[y <= 4.99], y[y > 4.99] = 1, 0\n\nclass _callback(keras.callbacks.Callback):\n  def on_epoch_end(self, epoch, logs={}):\n    if (epoch + 1) % 50 == 0:\n      self.model.optimizer.lr = self.model.optimizer.lr / 2\n\ncheckpoint_filepath = \'/tmp/checkpoint\'\n\nfold_no = 0 \n#StratifiedKFold\nfor train_index, val_index in StratifiedKFold(n_splits= 5, shuffle= True, random_state= seed).split(X_5_fold, y):\n\n  x_tr, x_val = X_5_fold[train_index], X_5_fold[val_index]\n  y_tr, y_val = y[train_index], y[val_index]\n\n\n\n  fold_no += 1\n\n  print(\'-------------------------------------------------------------------------------\')\n  print(\'

In [ ]:
'''
# == Provide average scores ==
print('-----------------------------' * 4)
print('Results per fold: ')
for i in range(0, len(ACCs)):
  print('-----------------------------' * 4)
  print(f'Fold {i+1} - ACC: {ACCs[i]}')
print('-----------------------------' * 4)
print('Average scores for all folds:')

print("\nACC:  %.0f (+/-) %.0f" % (np.mean(ACCs)*100 , np.std(ACCs)*100 ))


print('----------------------------' * 4)


tr_Acc = np.mean((
        CV_history[0].history['accuracy'][-1],
        CV_history[1].history['accuracy'][-1],
        CV_history[2].history['accuracy'][-1],
        CV_history[3].history['accuracy'][-1],
        CV_history[4].history['accuracy'][-1]))

print('----------------------------' * 4)

print("\nTrain ACC:   %.0f (+/-) %.0f" % (np.mean(tr_Acc)*100  , np.std(tr_Acc)*100  ))
'''

'\n# == Provide average scores ==\nprint(\'-----------------------------\' * 4)\nprint(\'Results per fold: \')\nfor i in range(0, len(ACCs)):\n  print(\'-----------------------------\' * 4)\n  print(f\'Fold {i+1} - ACC: {ACCs[i]}\')\nprint(\'-----------------------------\' * 4)\nprint(\'Average scores for all folds:\')\n\nprint("\nACC:  %.0f (+/-) %.0f" % (np.mean(ACCs)*100 , np.std(ACCs)*100 ))\n\n\nprint(\'----------------------------\' * 4)\n\n\ntr_Acc = np.mean((\n        CV_history[0].history[\'accuracy\'][-1],\n        CV_history[1].history[\'accuracy\'][-1],\n        CV_history[2].history[\'accuracy\'][-1],\n        CV_history[3].history[\'accuracy\'][-1],\n        CV_history[4].history[\'accuracy\'][-1]))\n\nprint(\'----------------------------\' * 4)\n\nprint("\nTrain ACC:   %.0f (+/-) %.0f" % (np.mean(tr_Acc)*100  , np.std(tr_Acc)*100  ))\n'

In [ ]:
#print(ACCs)

# **One Sub**

In [ ]:
'''
model = create_model()

class _callback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if (epoch + 1) % 15 == 0:
      self.model.optimizer.lr = self.model.optimizer.lr / 2

checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
      filepath=checkpoint_filepath,
      save_weights_only=True,
      monitor='val_accuracy',
      verbose= 1,
      mode='max',
      save_best_only=True)

model.compile(optimizer= keras.optimizers.Adam(1e-4),
              loss= 'binary_crossentropy',
              metrics= ['accuracy'])

X_train, X_test, Y_train, Y_test = read_data(X_List, Y, 2)
X_train  = np.reshape(X_train , (X_train.shape[0] , 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

model.fit(X_train, Y_train, epochs=50,
          validation_data=(X_test, Y_test),
          batch_size= 16,
          callbacks= [model_checkpoint_callback])
          
# X_train.shape, X_test.shape, Y_train.shape, Y_test.shape
model.load_weights(checkpoint_filepath)

'''

"\nmodel = create_model()\n\nclass _callback(keras.callbacks.Callback):\n  def on_epoch_end(self, epoch, logs={}):\n    if (epoch + 1) % 15 == 0:\n      self.model.optimizer.lr = self.model.optimizer.lr / 2\n\ncheckpoint_filepath = '/tmp/checkpoint'\nmodel_checkpoint_callback = keras.callbacks.ModelCheckpoint(\n      filepath=checkpoint_filepath,\n      save_weights_only=True,\n      monitor='val_accuracy',\n      verbose= 1,\n      mode='max',\n      save_best_only=True)\n\nmodel.compile(optimizer= keras.optimizers.Adam(1e-4),\n              loss= 'binary_crossentropy',\n              metrics= ['accuracy'])\n\nX_train, X_test, Y_train, Y_test = read_data(X_List, Y, 2)\nX_train  = np.reshape(X_train , (X_train.shape[0] , 1, X_train.shape[1]))\nX_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))\n\nmodel.fit(X_train, Y_train, epochs=50,\n          validation_data=(X_test, Y_test),\n          batch_size= 16,\n          callbacks= [model_checkpoint_callback])\n          \n#

# **ALL Subject**

In [ ]:
results = []

for subject_ in range(19):

  start_time_w_model = time.time() 

  model = create_model()


  class _callback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
      if (epoch + 1) % 15 == 0:
        self.model.optimizer.lr = self.model.optimizer.lr / 2

  checkpoint_filepath = '/tmp/checkpoint'
  model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        monitor='val_accuracy',
        verbose= 1,
        mode='max',
        save_best_only=True)



  # Compile Model
  model.compile(optimizer= keras.optimizers.Adam(1e-4),
                loss= 'binary_crossentropy',
                metrics= ['accuracy'])




  # Read Data
  X_train, X_test, Y_train, Y_test = read_data(X_List, Y, subject_)



  # Reshape Data
  X_train  = np.reshape(X_train , (X_train.shape[0] , 1, X_train.shape[1]))
  X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))



  # Time
  start_time_wo_model = time.time()



  # Fit Model
  model.fit(X_train, Y_train, epochs=50,
            validation_data=(X_test, Y_test),
            batch_size= 16,
            callbacks= [_callback(), model_checkpoint_callback]
            )
  
  

  # Updating to use the best model
  model.load_weights(checkpoint_filepath)

  y_pred = model.predict(X_test)
  y_pred[y_pred > 0.5], y_pred[y_pred < 0.5] = 1 , 0

  

  # Save Result
  elapsed_time_with_model = time.time() - start_time_w_model
  elapsed_time_without_model = time.time() - start_time_wo_model

  res = {
      "subject": f"{subject_:03}",
      "test_acc": model.evaluate(X_test, Y_test, verbose= 0)[1],
      "test_loss": model.evaluate(X_test, Y_test, verbose= 0)[0],
      "train_acc": model.evaluate(X_train, Y_train, verbose= 0)[1],
      "train_loss": model.evaluate(X_train, Y_train, verbose= 0)[0],
      "classification_res": classification_report(Y_test, y_pred),
      "time_with_model_creation": elapsed_time_with_model,
      "time_without_model_creation": elapsed_time_without_model
  }


  print(f"subj: {res.get('subject')}, test_acc: {res.get('test_acc')}, train_acc: {res.get('train_acc')}")
  print(f"Elapsed Times: {res.get('elapsed_time_without_model')}, {res.get('elapsed_time_without_model')}")
  
  results.append(res)



Epoch 1/50
102/102 [==============================] - ETA: 0s - loss: 1.0457 - accuracy: 0.5210
Epoch 00001: val_accuracy improved from -inf to 0.50000, saving model to /tmp/checkpoint
102/102 [==============================] - 20s 198ms/step - loss: 1.0457 - accuracy: 0.5210 - val_loss: 0.8645 - val_accuracy: 0.5000
Epoch 2/50
102/102 [==============================] - ETA: 0s - loss: 0.7974 - accuracy: 0.4870
Epoch 00002: val_accuracy did not improve from 0.50000
102/102 [==============================] - 20s 196ms/step - loss: 0.7974 - accuracy: 0.4870 - val_loss: 0.7591 - val_accuracy: 0.5000
Epoch 3/50
102/102 [==============================] - ETA: 0s - loss: 0.7458 - accuracy: 0.5086
Epoch 00003: val_accuracy did not improve from 0.50000
102/102 [==============================] - 20s 193ms/step - loss: 0.7458 - accuracy: 0.5086 - val_loss: 0.7363 - val_accuracy: 0.5000
Epoch 4/50
102/102 [==============================] - ETA: 0s - loss: 0.7287 - accuracy: 0.4833
Epoch 00004: va

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


subj: 001, test_acc: 0.5444444417953491, train_acc: 0.6098765134811401
Elapsed Times: None, None
Epoch 1/50
102/102 [==============================] - ETA: 0s - loss: 1.0758 - accuracy: 0.5093
Epoch 00001: val_accuracy improved from -inf to 0.50000, saving model to /tmp/checkpoint
102/102 [==============================] - 20s 200ms/step - loss: 1.0758 - accuracy: 0.5093 - val_loss: 0.8984 - val_accuracy: 0.5000
Epoch 2/50
102/102 [==============================] - ETA: 0s - loss: 0.8361 - accuracy: 0.4944
Epoch 00002: val_accuracy did not improve from 0.50000
102/102 [==============================] - 20s 194ms/step - loss: 0.8361 - accuracy: 0.4944 - val_loss: 0.7913 - val_accuracy: 0.5000
Epoch 3/50
102/102 [==============================] - ETA: 0s - loss: 0.7792 - accuracy: 0.4963
Epoch 00003: val_accuracy did not improve from 0.50000
102/102 [==============================] - 20s 194ms/step - loss: 0.7792 - accuracy: 0.4963 - val_loss: 0.7633 - val_accuracy: 0.5000
Epoch 4/50
102

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


subj: 002, test_acc: 0.6000000238418579, train_acc: 0.5228394865989685
Elapsed Times: None, None
Epoch 1/50
102/102 [==============================] - ETA: 0s - loss: 1.0432 - accuracy: 0.5216
Epoch 00001: val_accuracy improved from -inf to 0.50000, saving model to /tmp/checkpoint
102/102 [==============================] - 21s 205ms/step - loss: 1.0432 - accuracy: 0.5216 - val_loss: 0.8647 - val_accuracy: 0.5000
Epoch 2/50
102/102 [==============================] - ETA: 0s - loss: 0.8001 - accuracy: 0.5000
Epoch 00002: val_accuracy did not improve from 0.50000
102/102 [==============================] - 20s 191ms/step - loss: 0.8001 - accuracy: 0.5000 - val_loss: 0.7644 - val_accuracy: 0.5000
Epoch 3/50
102/102 [==============================] - ETA: 0s - loss: 0.7494 - accuracy: 0.5000
Epoch 00003: val_accuracy did not improve from 0.50000
102/102 [==============================] - 20s 191ms/step - loss: 0.7494 - accuracy: 0.5000 - val_loss: 0.7362 - val_accuracy: 0.5000
Epoch 4/50
102

...

subj: 003, test_acc: 0.5, train_acc: 0.5
Elapsed Times: None, None
Epoch 1/50
102/102 [==============================] - ETA: 0s - loss: 1.0569 - accuracy: 0.5142
Epoch 00001: val_accuracy improved from -inf to 0.50000, saving model to /tmp/checkpoint
102/102 [==============================] - 21s 205ms/step - loss: 1.0569 - accuracy: 0.5142 - val_loss: 0.8779 - val_accuracy: 0.5000
Epoch 2/50
102/102 [==============================] - ETA: 0s - loss: 0.8149 - accuracy: 0.4963
Epoch 00002: val_accuracy did not improve from 0.50000
102/102 [==============================] - 20s 198ms/step - loss: 0.8149 - accuracy: 0.4963 - val_loss: 0.7755 - val_accuracy: 0.5000
Epoch 3/50
102/102 [==============================] - ETA: 0s - loss: 0.7661 - accuracy: 0.5019
Epoch 00003: val_accuracy did not improve from 0.50000
102/102 [==============================] - 21s 205ms/step - loss: 0.7661 - accuracy: 0.5019 - val_loss: 0.7522 - val_accuracy: 0.5000
Epoch 4/50
102/102 [========================

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


subj: 004, test_acc: 0.5666666626930237, train_acc: 0.5555555820465088
Elapsed Times: None, None
Epoch 1/50
102/102 [==============================] - ETA: 0s - loss: 1.0333 - accuracy: 0.5160
Epoch 00001: val_accuracy improved from -inf to 0.50000, saving model to /tmp/checkpoint
102/102 [==============================] - 21s 202ms/step - loss: 1.0333 - accuracy: 0.5160 - val_loss: 0.8482 - val_accuracy: 0.5000
Epoch 2/50
102/102 [==============================] - ETA: 0s - loss: 0.7843 - accuracy: 0.5049
Epoch 00002: val_accuracy did not improve from 0.50000
102/102 [==============================] - 20s 194ms/step - loss: 0.7843 - accuracy: 0.5049 - val_loss: 0.7484 - val_accuracy: 0.5000
Epoch 3/50
102/102 [==============================] - ETA: 0s - loss: 0.7377 - accuracy: 0.4938
Epoch 00003: val_accuracy did not improve from 0.50000
102/102 [==============================] - 20s 192ms/step - loss: 0.7377 - accuracy: 0.4938 - val_loss: 0.7279 - val_accuracy: 0.5000
Epoch 4/50
102

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


subj: 005, test_acc: 0.5, train_acc: 0.5
Elapsed Times: None, None
Epoch 1/50
102/102 [==============================] - ETA: 0s - loss: 1.0661 - accuracy: 0.5062
Epoch 00001: val_accuracy improved from -inf to 0.52222, saving model to /tmp/checkpoint
102/102 [==============================] - 21s 201ms/step - loss: 1.0661 - accuracy: 0.5062 - val_loss: 0.8943 - val_accuracy: 0.5222
Epoch 2/50
102/102 [==============================] - ETA: 0s - loss: 0.8328 - accuracy: 0.4907
Epoch 00002: val_accuracy did not improve from 0.52222
102/102 [==============================] - 20s 192ms/step - loss: 0.8328 - accuracy: 0.4907 - val_loss: 0.7896 - val_accuracy: 0.5000
Epoch 3/50
102/102 [==============================] - ETA: 0s - loss: 0.7754 - accuracy: 0.5062
Epoch 00003: val_accuracy did not improve from 0.52222
102/102 [==============================] - 20s 196ms/step - loss: 0.7754 - accuracy: 0.5062 - val_loss: 0.7599 - val_accuracy: 0.5000
Epoch 4/50
102/102 [========================

In [ ]:
res.get('subject')

In [ ]:
print(f"Elapsed Times: {res.get('time_with_model_creation')}, {res.get('time_without_model_creation')}")

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Lstm_Val_file02.txt', 'w') as filehandle:
  filehandle.writelines("%s\n" % place for place in results)


#with open('drive/My Drive/fMRI/listfile.txt', 'w') as filehandle:
    #filehandle.writelines("%s\n" % place for place in results)  

In [ ]:
np.mean([_.get('train_acc') for _ in results]), np.std([_.get('train_acc') for _ in results])

In [ ]:
np.mean([_.get('test_acc') for _ in results]), np.std([_.get('test_acc') for _ in results])